# Install Library yang dibutuhkan

In [1]:
pip install tfx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.5/173.5 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
IN

# Import Library

In [1]:
# Import library
import os
from typing import Text
from absl import logging
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner

Melakukan set variabel seperti pipeline name, path untuk menyimpan output, path module, dan banyak lainnya.

In [2]:
# Nama pipeline
PIPELINE_NAME = "heart-disease-pipeline"

# Pipeline inputs
DATA_ROOT = "/content/data"
TRANSFORM_MODULE_FILE = "/content/modules/transform.py"
TRAINER_MODULE_FILE = "/content/modules/trainer.py"
COMPONENTS_MODULE_FILE = "/content/modules/components.py"

# Pipeline outputs
OUTPUT_BASE = "output"
serving_model_dir = os.path.join(OUTPUT_BASE, 'serving_model')
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

Pembuatan pipeline component module file menggunakan magic command. Pipeline terdiri dari:

1. CsvExampleGen
2. StatisticsGen
3. SchemaGen
4. ExampleValidator
5. Transform
6. Trainer
7. Evaluator
8. Pusher

Komponen trainer sudah menggunakan komponen tuner. Pusher akan melakukan push model jika melebihi syarat dari BinaryAccuracy 0.5

# Membuat Pipeline Components

In [3]:
%%writefile {COMPONENTS_MODULE_FILE}

# Import library
import os
import tensorflow_model_analysis as tfma
from tfx.components import (
    CsvExampleGen,
    StatisticsGen,
    SchemaGen,
    ExampleValidator,
    Transform,
    Trainer,
    Evaluator,
    Pusher
)
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.types import Channel
from tfx.dsl.components.common.resolver import Resolver
from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import (
    LatestBlessedModelStrategy)

# Fungsi untuk melakukan inisialisasi components
def init_components(config):

    """Returns tfx components for the pipeline.

    Args:
        data_dir (str): Directory containing the dataset.
        transform_module (str): Path to the transform module.
        tuner_module (str): Path to the tuner module.
        training_module (str): Path to the training module.
        training_steps (int): Number of training steps.
        eval_steps (int): Number of evaluation steps.
        serving_model_dir (str): Directory to save the serving

    Returns:
        components: Tuple of TFX components.
    """

    # Membagi dataset dengan perbandingan 8:2
    output = example_gen_pb2.Output(
        split_config = example_gen_pb2.SplitConfig(splits=[
            example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
            example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
        ])
    )

    # Komponen example gen
    example_gen = CsvExampleGen(
        input_base=config["DATA_ROOT"],
        output_config=output
    )

    # Komponen statistics gen
    statistics_gen = StatisticsGen(
        examples=example_gen.outputs["examples"]
    )

    # Komponen schema gen
    schema_gen = SchemaGen(
        statistics=statistics_gen.outputs["statistics"]
    )

    # Komponen example validator
    example_validator = ExampleValidator(
        statistics=statistics_gen.outputs['statistics'],
        schema=schema_gen.outputs['schema']
    )

    # Komponen transform. Menggunakan module transform.py
    transform  = Transform(
        examples=example_gen.outputs['examples'],
        schema= schema_gen.outputs['schema'],
        module_file=os.path.abspath(config["transform_module"])
    )

    # Komponen trainer. Menggunakan module trainer.py
    trainer  = Trainer(
        module_file=os.path.abspath(config["training_module"]),
        examples = transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        schema=schema_gen.outputs['schema'],
        train_args=trainer_pb2.TrainArgs(
            splits=['train'],
            num_steps=config["training_steps"]),
        eval_args=trainer_pb2.EvalArgs(
            splits=['eval'],
            num_steps=config["eval_steps"])
    )

    # Komponen model resolver
    model_resolver = Resolver(
        strategy_class= LatestBlessedModelStrategy,
        model = Channel(type=Model),
        model_blessing = Channel(type=ModelBlessing)
    ).with_id('Latest_blessed_model_resolver')

    metrics_specs = [
        tfma.MetricsSpec(metrics=[
                tfma.MetricConfig(class_name='AUC'),
                tfma.MetricConfig(class_name="Precision"),
                tfma.MetricConfig(class_name="Recall"),
                tfma.MetricConfig(class_name="ExampleCount"),
                tfma.MetricConfig(class_name='BinaryAccuracy',
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={'value':0.8}),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={'value':0.0001})
                        )
                )
            ])
    ]


    eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='target')],  # Ensure 'target' is the correct label
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=metrics_specs
    )


    # Komponen evaluator
    evaluator = Evaluator(
        examples=example_gen.outputs['examples'],
        model=trainer.outputs['model'],
        baseline_model=model_resolver.outputs['model'],
        eval_config=eval_config)

    # Komponen pusher
    pusher = Pusher(
        model=trainer.outputs["model"],
        model_blessing=evaluator.outputs["blessing"],
        push_destination=pusher_pb2.PushDestination(
            filesystem=pusher_pb2.PushDestination.Filesystem(
                base_directory=config["serving_model_dir"]
            )
        ),
    )

    # Mengembalikan semua komponen
    components = (
        example_gen,
        statistics_gen,
        schema_gen,
        example_validator,
        transform,
        trainer,
        model_resolver,
        evaluator,
        pusher
    )

    # Mengembalikan komponen
    return components

Overwriting /content/modules/components.py


# Data Transform

In [4]:
%%writefile {TRANSFORM_MODULE_FILE}

# Import library
import tensorflow as tf
import tensorflow_transform as tft

LABEL_KEY = "target"

# List of feature names and types
NUMERIC_FEATURES = ['age', 'ca', 'chol', 'oldpeak', 'thalach', 'trestbps']
CATEGORICAL_FEATURES = ['cp', 'exang', 'fbs', 'restecg', 'sex', 'slope', 'thal']

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features.

    Args:
        inputs: dictionary of raw input features.

    Returns:
        outputs: dictionary of transformed features.
    """
    outputs = {}

    # Filter out rows with invalid values for 'ca' and 'thal'
    valid_rows = tf.logical_and(
        tf.not_equal(inputs['ca'], 4),  # Exclude rows where `ca` is 4
        tf.not_equal(inputs['thal'], 0)  # Exclude rows where `thal` is 0
    )

    # Apply the valid row mask to all inputs
    filtered_inputs = {key: tf.boolean_mask(inputs[key], valid_rows) for key in inputs}

    # Normalize numeric features
    for feature in NUMERIC_FEATURES:
        outputs[transformed_name(feature)] = tft.scale_to_z_score(filtered_inputs[feature])

    # Label encode categorical features
    for feature in CATEGORICAL_FEATURES:
        outputs[transformed_name(feature)] = tft.compute_and_apply_vocabulary(filtered_inputs[feature])

    # Include the label
    outputs[LABEL_KEY] = filtered_inputs[LABEL_KEY]

    return outputs

Overwriting /content/modules/transform.py


Kode di atas adalah sebuah modul pemrosesan data untuk pra-pemrosesan fitur pada dataset menggunakan TensorFlow Transform (TFT). Berikut adalah penjelasan singkat dari proses yang dilakukan:

1. Fitur Numerik: Fitur numerik seperti usia, kolesterol, dan tekanan darah diubah agar memiliki distribusi yang lebih baik dengan melakukan normalisasi menggunakan teknik z-score (standarisasi).

2. Fitur Kategorikal: Fitur kategorikal seperti jenis kelamin dan hasil tes kesehatan lainnya diberi representasi angka (label encoding) berdasarkan frekuensi kemunculan kategori tersebut dalam dataset.

3. Filter Data Tidak Valid: Beberapa baris data yang memiliki nilai tidak valid, seperti nilai ca yang bernilai 4 atau thal yang bernilai 0, dihapus dari dataset.

4. Fungsi Utama: Fungsi preprocessing_fn mengatur semua langkah pemrosesan data ini dengan menerima data mentah sebagai input dan mengembalikan data yang telah diproses sesuai dengan transformasi yang telah ditentukan.

Proses ini penting untuk membersihkan dan menyiapkan data sebelum digunakan dalam model machine learning.

# Model Development

In [5]:
%%writefile {TRAINER_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft
from tfx.components.trainer.fn_args_utils import FnArgs
from keras.utils import plot_model
import os

# Definisikan nama label
LABEL_KEY = "target"

# Daftar fitur numerik dan kategorikal
NUMERIC_FEATURES = ['age', 'ca', 'chol', 'oldpeak', 'thalach', 'trestbps']
CATEGORICAL_FEATURES = ['cp', 'exang', 'fbs', 'restecg', 'sex', 'slope', 'thal']

def transformed_name(key):
    """Menambahkan suffix '_xf' pada nama fitur yang telah ditransformasi"""
    return key + "_xf"

# Fungsi untuk membaca data yang telah di-compress
def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

# Fungsi input untuk mempersiapkan dataset
def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=64):
    # Mendapatkan feature_spec untuk fitur yang sudah ditransformasi
    transform_feature_spec = tf_transform_output.transformed_feature_spec().copy()

    # Membaca data dalam bentuk batch
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key='target'  # Menggunakan 'target' sebagai label key yang benar
    )

    # Fungsi untuk format data (menyesuaikan label dan fitur)
    def format_data(features, labels):
        labels = tf.reshape(labels, [-1, 1])  # Bentuk label sesuai dengan output
        return features, labels

    return dataset.map(format_data)


# Membangun model
def model_builder():
    inputs = []

    # Menambahkan input layer untuk setiap fitur
    for feature in NUMERIC_FEATURES + CATEGORICAL_FEATURES:
        inputs.append(tf.keras.Input(shape=(1,), name=transformed_name(feature)))

    # Menggabungkan input layer
    x = tf.keras.layers.Concatenate()(inputs)

    # Hidden layers
    x = tf.keras.layers.Dense(8, activation='relu')(x)
    x = tf.keras.layers.Dense(16, activation='relu')(x)
    x = tf.keras.layers.Dense(16, activation='relu')(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)

    # Output layer untuk klasifikasi
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    # Membuat model
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    # Menyusun model dengan optimizer dan loss function
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    model.summary()

    return model

# Fungsi untuk menyajikan TF examples
def _get_serve_tf_examples_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)

        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)

        return model(transformed_features)

    return serve_tf_examples_fn

# Fungsi untuk mendapatkan signature dari fitur transformasi
def _get_transform_features_signature(model, tf_transform_output):
    model.tft_layer_eval = tf_transform_output.transform_features_layer()

    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
    ])
    def transform_features_fn(serialized_tf_example):
        raw_feature_spec = tf_transform_output.raw_feature_spec()
        raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
        transformed_features = model.tft_layer_eval(raw_features)
        return transformed_features

    return transform_features_fn

# Fungsi utama untuk menjalankan pelatihan
def run_fn(fn_args: FnArgs):
    # Menginisialisasi tf_transform_output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    # Membaca dataset pelatihan dan evaluasi
    train_dataset = input_fn(fn_args.train_files, tf_transform_output, num_epochs=20)
    eval_dataset = input_fn(fn_args.eval_files, tf_transform_output, num_epochs=1)

    # Membangun model
    model = model_builder()

    # Melatih model
    model.fit(train_dataset, epochs=20, validation_data=eval_dataset)

    # Menyimpan model dengan signatures untuk serving
    signatures = {
        'serving_default': _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
            tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')),
        'transform_features': _get_transform_features_signature(model, tf_transform_output),
    }

    # Menyimpan model yang telah dilatih
    tf.saved_model.save(model, fn_args.serving_model_dir, signatures=signatures)

    plot_model(
        model,
        to_file='images/model_plot.png',
        show_shapes=True,
        show_layer_names=True
    )

Overwriting /content/modules/trainer.py


# Melakukan Inisialisasi Local Pipeline.

In [6]:
def init_local_pipeline(
    components, pipeline_root: Text
) -> pipeline.Pipeline:

    logging.info(f"Pipeline root set to: {pipeline_root}")
    beam_args = [
        "--direct_running_mode=multi_processing",
        # 0 auto-detect based on on the number of CPUs available
        # during execution time.
        "----direct_num_workers=0"
    ]

    return pipeline.Pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=pipeline_root,
        components=components,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            metadata_path
        ),
        beam_pipeline_args=beam_args
    )

# Menjalankan Pipeline Menggunakan Apache Beam.

In [7]:
from modules.components import init_components

logging.set_verbosity(logging.INFO)

config = {
    "DATA_ROOT": DATA_ROOT,
    "training_module": TRAINER_MODULE_FILE,
    "transform_module": TRANSFORM_MODULE_FILE,
    "training_steps": 1000,
    "eval_steps": 250,
    "serving_model_dir": serving_model_dir,
}

components = init_components(config)

pipeline = init_local_pipeline(components, pipeline_root)
BeamDagRunner().run(pipeline=pipeline)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Pipeline root set to: output/heart-disease-pipeline
INFO:absl:Generating ephemeral wheel package for '/content/modules/transform.py' (including modules: ['transform', 'trainer', 'components']).
INFO:absl:User module package has hash fingerprint version c3d3aea9d3172fa46c84edccb5e51a69a35f4a1886bdaa8bfe8e9a11ae49595a.
INFO:absl:Executing: ['/usr/bin/python3', '/tmp/tmpi90y2fhu/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp_l0djqs8', '--dist-dir', '/tmp/tmp5xa_c4da']
INFO:absl:Successfully built user code wheel distribution at 'output/heart-disease-pipeline/_wheels/tfx_user_code_Transform-0.0+c3d3aea9d3172fa46c84edccb5e51a69a35f4a1886bdaa8bfe8e9a11ae49595a-py3-none-any.whl'; target user module is 'transform'.
INFO:absl:Full user module path is 'transform@output/heart-

INFO:absl:Node CsvExampleGen depends on [].
INFO:absl:Node CsvExampleGen is scheduled.
INFO:absl:Node Latest_blessed_model_resolver depends on [].
INFO:absl:Node Latest_blessed_model_resolver is scheduled.
INFO:absl:Node StatisticsGen depends on ['Run[CsvExampleGen]'].
INFO:absl:Node StatisticsGen is scheduled.
INFO:absl:Node SchemaGen depends on ['Run[StatisticsGen]'].
INFO:absl:Node SchemaGen is scheduled.
INFO:absl:Node ExampleValidator depends on ['Run[SchemaGen]', 'Run[StatisticsGen]'].
INFO:absl:Node ExampleValidator is scheduled.
INFO:absl:Node Transform depends on ['Run[CsvExampleGen]', 'Run[SchemaGen]'].
INFO:absl:Node Transform is scheduled.
INFO:absl:Node Trainer depends on ['Run[SchemaGen]', 'Run[Transform]'].
INFO:absl:Node Trainer is scheduled.
INFO:absl:Node Evaluator depends on ['Run[CsvExampleGen]', 'Run[Latest_blessed_model_resolver]', 'Run[Trainer]'].
INFO:absl:Node Evaluator is scheduled.
INFO:absl:Node Pusher depends on ['Run[Evaluator]', 'Run[Trainer]'].
INFO:absl

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ age_xf (InputLayer)       │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ ca_xf (InputLayer)        │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ chol_xf (InputLayer)      │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ oldpeak_xf (InputLayer)   │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ thalach_xf (InputLayer)   │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ trestbps_xf (InputLayer)  │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cp_xf (InputLayer)        │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ exang_xf (InputLayer)     │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ fbs_xf (InputLayer)       │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ restecg_xf (InputLayer)   │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sex_xf (InputLayer)       │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ slope_xf (InputLayer)     │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ thal_xf (InputLayer)      │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 13)             │              0 │ age_xf[0][0],          │
│                           │                        │                │ ca_xf[0][0],           │
│                           │                        │                │ chol_xf[0][0],         │
│                           │                        │                │ oldpeak_xf[0][0],      │
│                           │                        │                │ thalach_xf[0][0],      │
│                           │                        │                │ trestbps_xf[0][0],     │
│                           │                        │                │ cp_xf[0][0],           │
│                           │                        │                │ exang_xf[0][0],        │
│                           │                        │                │ fbs_xf[0][0],          │
│                           │                        │                │ restecg_xf[0][0],      │
│                           │                        │                │ sex_xf[0][0],          │
│                           │                        │                │ slope_xf[0][0],        │
│                      

 Total params: 1,105 (4.32 KB)

...

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
     76/Unknown 2s 3ms/step - accuracy: 0.6027 - loss: 0.6233

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6044 - loss: 0.6221 - val_accuracy: 0.7843 - val_loss: 0.5106
Epoch 2/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8055 - loss: 0.4605 - val_accuracy: 0.8431 - val_loss: 0.3871
Epoch 3/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8620 - loss: 0.3426 - val_accuracy: 0.8824 - val_loss: 0.3510
Epoch 4/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8836 - loss: 0.2843 - val_accuracy: 0.8824 - val_loss: 0.3673
Epoch 5/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8949 - loss: 0.2466 - val_accuracy: 0.8824 - val_loss: 0.4224
Epoch 6/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9213 - loss: 0.2113 - val_accuracy: 0.8824 - val_loss: 0.5200
Epoch 7/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9323 - loss: 0.1764 - val_accuracy: 0.8824 - val_loss: 0.5980
Epoch 8/20
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9425 - loss: 0.1475 - val_accuracy: 0.8824 - val_loss: 0.6647
Epo

INFO:absl:Feature age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature ca has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature chol has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature cp has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature exang has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature fbs has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature oldpeak has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature restecg has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature sex has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature slope has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature target has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature thal has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature thalach has a shape dim {
  siz

In [8]:
# Tulis isi Dockerfile
dockerfile_content = """
FROM tensorflow/serving:latest

COPY ./output/serving_model /models/deploy
COPY ./config /model_config
ENV MODEL_NAME=deploy

ENV MONITORING_CONFIG="/model_config/prometheus.config"
ENV PORT=8501

RUN echo '#!/bin/bash \n\n\
env \n\
tensorflow_model_server --port=8500 --rest_api_port=${PORT} \
--model_name=${MODEL_NAME} --model_base_path=${MODEL_BASE_PATH}/${MODEL_NAME} \
--monitoring_config_file=${MONITORING_CONFIG} \
"$@"' > /usr/bin/tf_serving_entrypoint.sh \
&& chmod +x /usr/bin/tf_serving_entrypoint.sh
"""

# Simpan sebagai file Dockerfile
with open("Dockerfile", "w") as f:
    f.write(dockerfile_content)

print("Dockerfile has been written.")

Dockerfile has been written.


In [9]:
# Tulis isi Dockerfile
dockerfile_content = """
FROM prom/prometheus:latest

COPY prometheus.yml /etc/prometheus/prometheus.yml
"""

# Simpan sebagai file Dockerfile
with open("/content/monitoring/Dockerfile", "w") as f:
    f.write(dockerfile_content)

print("Dockerfile has been written.")

Dockerfile has been written.


In [10]:
# Tulis isi yml
yml_content = """
global:
  scrape_interval: 5s
  evaluation_interval: 5s
  external_labels:
    monitor: "tf-serving-monitor"

scrape_configs:
  - job_name: "prometheus"
    scrape_interval: 5s
    metrics_path: /monitoring/prometheus/metrics
    scheme: "https"
    static_configs:
      - targets: ['proyek-akhir-mlops-production.up.railway.app']
"""

# Simpan sebagai file Dockerfile
with open("/content/monitoring/prometheus.yml", "w") as f:
    f.write(yml_content)

print("prometheus has been written.")

prometheus has been written.


In [11]:
# Tulis isi config
config_content = """
prometheus_config {
   enable: true,
   path: "/monitoring/prometheus/metrics"
}
"""

# Simpan sebagai file Dockerfile
with open("/content/config/prometheus.config", "w") as f:
    f.write(config_content)

print("prometheus has been written.")

prometheus has been written.


In [12]:
import shutil

# Path to the directory you want to zip
dir_path = '/content'

# Path where the zip file will be saved
output_zip_path = '/content/data.zip'

# Create a zip file
shutil.make_archive(output_zip_path.replace('.zip', ''), 'zip', dir_path)

'/content/data.zip'